In [31]:
# import dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [10]:
merged_nb_df = pd.read_csv("merged_nb_df.csv")
pd.set_option('display.max_columns', None)
merged_nb_df.head()

,size_l,og,fg,abv,ibu,color,boilsize,boiltime,boilgravity,efficiency,mashthickness,pitchrate,primarytemp,review_taste,beer_abv,sugarscale_Plato,sugarscale_Specific Gravity,brewmethod_All Grain,brewmethod_BIAB,brewmethod_Partial Mash,brewmethod_extract
0,18.93,1.082,1.013,9.10,0.00,4.10,21.58,60.0,0.000,72.0,0.0,0.00,0.00,4.103825,7.6,0.0,1.0,1.0,0.0,0.0,0.0
1,7.57,1.076,1.014,8.10,33.51,3.68,8.71,60.0,1.066,75.0,0.0,0.00,0.00,4.103825,7.6,0.0,1.0,1.0,0.0,0.0,0.0
2,20.82,1.043,1.011,4.14,13.22,17.67,28.39,90.0,0.000,85.0,0.0,1.75,12.22,3.028652,4.4,0.0,1.0,1.0,0.0,0.0,0.0
3,22.71,1.069,1.017,6.75,58.76,40.00,29.34,90.0,1.053,70.0,0.0,0.00,0.00,4.265971,6.4,0.0,1.0,1.0,0.0,0.0,0.0
4,22.71,1.068,1.018,6.53,73.40,40.00,27.44,90.0,1.056,65.0,1.3,1.25,18.89,4.265971,6.4,0.0,1.0,1.0,0.0,0.0,0.0


In [17]:
# What percentage of beers scored 4 or above?
len(merged_nb_df[(merged_nb_df["review_taste"]>=4)]) / len(merged_nb_df) * 100

34.77543538038497

In [19]:
# Since beers with a rating of four or above is the top 34% of beers, let's go with those as "good" beers.
# We need a new category "good beer" with a 1 for good and 0 for bad
merged_nb_df["good_beer"]=0

In [25]:
# for all the "good beers", put a 1 in the good_beer column
merged_nb_df.loc[merged_nb_df["review_taste"].abs()>=4, "good_beer"] = 1

In [26]:
merged_nb_df.head()

,size_l,og,fg,abv,ibu,color,boilsize,boiltime,boilgravity,efficiency,mashthickness,pitchrate,primarytemp,review_taste,beer_abv,sugarscale_Plato,sugarscale_Specific Gravity,brewmethod_All Grain,brewmethod_BIAB,brewmethod_Partial Mash,brewmethod_extract,good_beer
0,18.93,1.082,1.013,9.10,0.00,4.10,21.58,60.0,0.000,72.0,0.0,0.00,0.00,4.103825,7.6,0.0,1.0,1.0,0.0,0.0,0.0,1
1,7.57,1.076,1.014,8.10,33.51,3.68,8.71,60.0,1.066,75.0,0.0,0.00,0.00,4.103825,7.6,0.0,1.0,1.0,0.0,0.0,0.0,1
2,20.82,1.043,1.011,4.14,13.22,17.67,28.39,90.0,0.000,85.0,0.0,1.75,12.22,3.028652,4.4,0.0,1.0,1.0,0.0,0.0,0.0,0
3,22.71,1.069,1.017,6.75,58.76,40.00,29.34,90.0,1.053,70.0,0.0,0.00,0.00,4.265971,6.4,0.0,1.0,1.0,0.0,0.0,0.0,1
4,22.71,1.068,1.018,6.53,73.40,40.00,27.44,90.0,1.056,65.0,1.3,1.25,18.89,4.265971,6.4,0.0,1.0,1.0,0.0,0.0,0.0,1


In [27]:
beer_class = merged_nb_df.drop(columns = ["review_taste"])

In [28]:
# Split our preprocessed data into our features and target arrays
y = beer_class.good_beer.values
X = beer_class.drop(columns = ["good_beer"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5, stratify = y)

In [30]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [36]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=300, random_state=5)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest predictive accuracy: 0.615
